# Parse deWikipedia Vorlage:HOV und bereite Wikidata-Import vor

## Dieses Skript liefert folgendes Ergebnis:

1. Eine Tabelle aller Orte mit HOV-Vorlage in der DeWikipedia, mit enstprechendem Link nach Wikidata, HOV-Code, Bezeichnung in HOV, Vermerkung ob Eintrag in Wikidata bereits besteht und Levenshtein-Distanz
2. Einen TSV-Import für QuickStatements für alle Einträge mit Distanz=0

## Folgende Schritte durchläuft das Skript

1. API-Call dewikipedia.org
  1. Alle Pages mit Vorlage:HOV
  2. Parsen der Wikidata_ID aller Pages mit Vorlage:HOV
  3. Parsen des Wikitextes aller Pages mit Vorlage:HOV
2. Auslesen der Vorlage `{{HOV|Parameter}}` und Speichern des Parameters
3. Check ob Wikipedia-Seite überhaupt eine Siedlung (oder Unterklasse) beschreibt (Sparql-Query)
4. Laden der HOV_Website zum Auslesen des HOV-Codes und der Ansetzungsform des Ortsnamens 
5. Berechnung der Levenshtein-Distanz zwischen Wikipedia-Name und Parameterwert in der Vorlage
6. Erstellung des [QuickStatements-Importstrings](hov_dewikiqs.tsv) bei Datensätzen mit Distanz=0
7. Export aller Werte in eine [TSV-Datei](hovindewiki.tsv).

In [ ]:
import requests
import re
import json
from lxml import html
import urllib.parse
import urllib.request, json
import os
from SPARQLWrapper import SPARQLWrapper, JSON
from Levenshtein import distance
import time

#SPARQL-Params
endpoint_url = "https://query.wikidata.org/sparql"

#Mediawiki-Params
S = requests.Session()
URL = "https://de.wikipedia.org/w/api.php"
    #https://de.wikipedia.org/w/api.php?action=query&list=embeddedin&eititle=Template:HOV&eilimit=500&format=json
PARAMS = {
    'action': "query",
    'list': "embeddedin",
    'eititle': "Template:HOV",
    'eilimit': 10,
    'format': "json",
    'einamespace':0
}


def parseWikiAPI(URL,PARAMS):
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA

def getWikibaseIDviaMwapi(pagetitle,URL):
    param_wbase = { 'action' : 'query', 'prop' : 'pageprops', 'ppprop': 'wikibase_item',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    try:
        for page in DATA["query"]["pages"]:
            #print(page)
            try:
                return DATA["query"]["pages"][page]["pageprops"]["wikibase_item"]
            except KeyError:
                pass
    except KeyError:
        pass

def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url, agent='User:Mfchris84 Parsing Vorlage:HOV')
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


data = parseWikiAPI(URL,PARAMS)

try:
    os.remove('hovindewiki.tsv')
    os.remove('hov_dewikiqs.tsv')
except:
    pass

f = open('hovindewiki.tsv','a')
fqs = open('hov_dewikiqs.tsv','a')
i = 0

print("No.\tpageid\tqid\tdeWikiTitle\tHOV-String\tLevenshtein\tHovCodeinWD\tHovCodeParsed\tHOV_String_StatedAs\tZeroIsNotaSiedlung", file=f)
print("No.\tpageid\tqid\tdeWikiTitle\tHOV-String\tLevenshtein\tHovCodeinWD\tHovCodeParsed\tHOV_String_StatedAs\tZeroIsNotaSiedlung")


#https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q3778&props=claims
query = """SELECT * WHERE { ?item wdt:P7046 ?HOV_Code. }"""
results = get_results(endpoint_url, query)

#Run through the mediawiki-API request about all deWikipedia-Pages where Vorlage:HOV is embedded
try:
    while data["query"]:
        for pages in data["query"]["embeddedin"]:
            #Parse the Wikitext of the Page
            param_text = { 'action' : 'query', 'prop' : 'revisions', 'rvprop': 'content',
                     'rvlimit': 1, 'pageids': pages["pageid"],'format': "json" }
            wikitext = parseWikiAPI(URL,param_text)
            for page in wikitext["query"]["pages"]:
                test_str = wikitext["query"]["pages"][page]["revisions"][0]["*"]
                
                #Parse Vorlage:HOV
                hov_str = ""

                #a wenn {{HOV}} ohne Param vorkommt, dann muss PageName verwendet werden
                regex = r"{{HOV}}"
                matches = re.finditer(regex, test_str, re.MULTILINE)
                for matchNum, match in enumerate(matches, start=1):
                    hov_str = pages["title"]
                    
                #b trifft a nicht zu, dann Suche nach Vorlage:HOV mit Parameter
                if hov_str == "":
                    regex = r"{{HOV\|(.*?)(\||})"
                    matches = re.finditer(regex, test_str, re.MULTILINE)

                    for matchNum, match in enumerate(matches, start=1):
                        hov_str = match.group(1)
                    
                if hov_str == "":
                    hov_str = pages["title"]
                
                hov_str = re.sub("URL=","",hov_str)
                
                #Calc Levenshtein-Distance between Wikipedia-PageName and Hov-String
                lev_dist = distance(str(pages["title"]), hov_str)
                
                #Get Wikidata-ID of Wikipedia-Page
                qid = getWikibaseIDviaMwapi(pages["title"],URL)

                #Check if Wikipedia-Page describes a human settlement (to avoid parsing "Listen")    
                query_hs = """SELECT (COUNT(*) AS ?anzahl) WHERE {
                  wd:"""+qid+""" (wdt:P31/(wdt:P279*)) wd:Q486972;
                    wdt:P31 ?instanceOf.
                }"""    
                results_hs = get_results(endpoint_url, query_hs)
                time.sleep(1)
                
                for result in results_hs["results"]["bindings"]:
                    is_hs = result["anzahl"]["value"]
                
                #Look if HOV-Code alredy set in Wikidata
                hovc_wd = "None"
                for result in results["results"]["bindings"]:
                    if result["item"]["value"] == "http://www.wikidata.org/entity/"+str(qid):
                        hovc_wd = result["HOV_Code"]["value"]
                
                #Parse HOV-Code from hov.isgv.de
                hov_code_parsed = ""
                hov_str_statedas = ""
                if hovc_wd == "None":
                    #Go for hov.isgv.de/hov_str
                    hov_url = "https://hov.isgv.de/"+hov_str
                    page = requests.get(hov_url)
                    tree = html.fromstring(page.content)
                    #Parse HOV_Code 
                    hov_code = tree.xpath("//ul[@id='links']/text()")
                    #Parse additional h2 for "stated as"-Qualifier
                    hov_str_isgv = tree.xpath("//h2/strong/text()")
                    try:
                        hov_str_statedas = hov_str_isgv[0]
                    except:
                        pass
                        
                    try:
                        #print(hov_code[0])
                    
                        regex = r"HOV Code:\s([\d]+)"
                        matches = re.finditer(regex, hov_code[0], re.MULTILINE)
                        for matchNum, match in enumerate(matches, start=1):
                            hov_code_parsed = (match.group(1))
                            if (lev_dist == 0):
                                #Wenn Levenshtein zwischen Wikititel und HOV-Bezeichnung 0 - Identität gegeben - QS-Import vorbereiten:
                                print(qid+"\tP7046\t"+'"'+hov_code_parsed+'"\tP1932\t"'+hov_str_statedas+'"',file=fqs)
                    except:
                        pass                 

                
                print(str(i)+":\t"+str(pages["pageid"])+"\t"+qid+"\t"+str(pages["title"])+"\t"+hov_str+"\t"+str(lev_dist)+"\t"+hovc_wd+"\t"+hov_code_parsed+"\t"+hov_str_statedas+"\t"+is_hs,file=f)
                print(str(i)+":\t"+str(pages["pageid"])+"\t"+qid+"\t"+str(pages["title"])+"\t"+hov_str+"\t"+str(lev_dist)+"\t"+hovc_wd+"\t"+hov_code_parsed+"\t"+hov_str_statedas+"\t"+is_hs)
                
            i = i +1
        try:
        #print(DATA['continue']['cmcontinue'])
            PARAMS['eicontinue'] = data['continue']['eicontinue']
            data = parseWikiAPI(URL,PARAMS)
        except KeyError:
            break
except KeyError:
    pass      

f.close()

No.	pageid	qid	deWikiTitle	HOV-String	Levenshtein	HovCodeinWD	HovCodeParsed	HOV_String_StatedAs	ZeroIsNotaSiedlung
0:	1640	Q14819	Freiberg	Freiberg_(1)	4	16028			9
1:	2996	Q2079	Leipzig	Leipzig_(1)	4	4105			30
2:	3340	Q8738	Meißen	Meißen	0	10193			4
3:	8144	Q3778	Zwickau	Zwickau	0	22154			6
4:	12328	Q6477	Pirna	Pirna	0	11175			1
5:	12626	Q8920	Hellerau	Hellerau	0	8086			1
6:	14163	Q157095	Rothenburg/Oberlausitz	Rothenburg/O.L.	10	None	28096	Rothenburg/O.L.	4
7:	15893	Q8762	Radebeul	Radebeul	0	8184			4
8:	15927	Q8946	Pillnitz	Pillnitz	0	8168			1
9:	15947	Q71180	Rochlitz	Rochlitz_(1)	4	6117			4
10:	16976	Q4077	Görlitz	Görlitz_(1)	4	24018			16
11:	17322	Q14835	Bautzen	Bautzen	0	23007			4
12:	17803	Q12056	Oschatz	Oschatz	0	5142			4
13:	18004	Q5870	Freital	Freital	0	8246			30
14:	18011	Q6366	Pesterwitz	Pesterwitz_(Ober-)	8	None	8165	Pesterwitz (Ober-)	1
15:	18597	Q12058	Schkeuditz	Schkeuditz	0	31019			1
16:	19228	Q8248	Wilsdruff	Wilsdruff	0	10351			4
17:	19305	Q157200	Zittau	Zittau	0	29094	

140:	171591	Q436724	Altenberg (Erzgebirge)	Altenberg (1)	10	7145			4
141:	173309	Q12041	Bad Düben	Bad Düben	0	30046			4
142:	173513	Q502562	Oppach	Oppach	0	27093			1
143:	173537	Q502524	Eibau	Eibau_(Alt-)	7	None	27029	Eibau (Alt-)	4
144:	173542	Q1620424	Hirschfelde (Zittau)	Hirschfelde	9	None	29029	Hirschfelde	1
145:	173544	Q502508	Niedercunnersdorf	Niedercunnersdorf	0	27017			1
146:	173546	Q572527	Großschönau (Sachsen)	Großschönau	10	None	29076	Schönau, Groß-	4
147:	173551	Q502493	Hainewalde	Hainewalde	0	29020			1
148:	173553	Q502686	Leutersdorf (Oberlausitz)	Leutersdorf	14	None	29042	Leutersdorf	1
149:	173556	Q506142	Mittelherwigsdorf	Herwigsdorf,_Mittel-	16	None	29027	Herwigsdorf, Mittel-	1
150:	173579	Q544821	Jonsdorf	Jonsdorf,_Neu-	6	None	29035	Jonsdorf, Neu-	1
151:	173613	Q71137	Lunzenau	Lunzenau	0	6084			4
152:	174564	Q12050	Taucha	Taucha	0	4197			1
153:	174673	Q157480	Ebersbach/Sa.	Ebersbach/Sa._(Alt-)	7	None	27026	Ebersbach/Sa. (Alt-)	1
154:	175120	Q1069506	Chemnitz-Gablenz	Ga

268:	223431	Q93271	Obergurig	Obergurig	0	23113			1
269:	224593	Q93291	Schmölln-Putzkau	Schmölln-Putzkau	0	23342			1
270:	224762	Q93283	Puschwitz	Puschwitz_(1)	4	None	23222	Puschwitz (1)	1
271:	224774	Q93286	Radibor	Radibor	0	23232			1
272:	226218	Q2133428	Raun	Raun	0	19100			1
273:	227223	Q93279	Panschwitz-Kuckau	Panschwitz-Kuckau	0	26166			1
274:	228606	Q1804972	Langenau (Brand-Erbisdorf)	Langenau,_Ober-	16	None	16062	Langenau, Ober-	4
275:	228696	Q1167976	Kötzschenbroda	Kötzschenbroda	0	8102			1
276:	228763	Q250700	Jugel	Jugel (Ober-, Unter-)	16	None	21041	Jugel (Ober-, Unter-)	1
277:	229720	Q206739	Zitzschewig	Zitzschewig	0	8251			1
278:	229733	Q1988244	Niederlößnitz (Radebeul)	Lößnitz,_Nieder-_(1)	17	None	8125	Lößnitz, Nieder- (1)	1
279:	229743	Q1466523	Fürstenhain	Fürstenhain	0	8059			1
280:	229747	Q1416010	Lindenau (Radebeul)	Lindenau_(2)	9	None	8117	Lindenau (2)	1
281:	229756	Q1691815	Naundorf (Radebeul)	Naundorf_(11)	9	None	8141	Naundorf (11)	1
282:	229865	Q17112	Bernsdorf (Lan

399:	314581	Q703140	Uhyst (Spree)	Uhyst_(Klein-)	7	None	25101	Uhyst (Klein-)	1
400:	314663	Q506096	Königshain	Königshain_(1)	4	24025			1
401:	315269	Q502548	Groß Düben	Groß Düben	0	28012			1
402:	315278	Q544944	Hähnichen	Hähnichen	0	28023			1
403:	315323	Q703135	Klitten	Klitten	0	28041			4
404:	316328	Q58019	Sosa (Eibenstock)	Sosa	13	None	21078	Sosa	4
405:	316376	Q502675	Krauschwitz (Sachsen)	Krauschwitz	10	None	28049	Krauschwitz	1
406:	316389	Q502498	Kreba-Neudorf	Kreba-Neudorf	0	28143			1
407:	316528	Q502472	Neißeaue	Neißeaue	0	24076			1
408:	316532	Q502512	Rietschen	Rietschen	0	28094			1
409:	316540	Q506123	Markersdorf (Sachsen)	Markersdorf_(3)	8	None	24037	Markersdorf (3)	1
410:	316550	Q572576	Schleife (Ort)	Schleife	6	None	28104	Schleife	1
411:	316560	Q506128	Weißkeißel	Weißkeißel	0	28131			1
412:	316632	Q544870	Trebendorf	Trebendorf	0	28122			1
413:	316661	Q525896	Sohland am Rotstein	Sohland_a._Rotstein	3	None	27122	Sohland a. Rotstein	1
414:	317383	Q71117	Großschirma	Großschirma

529:	348615	Q46757	Grünbach (Sachsen)	Grünbach,_Höhenluftkurort	16	None	13033	Grünbach, Höhenluftkurort	1
530:	348654	Q695636	Hammerbrücke	Hammerbrücke	0	13036			1
531:	348765	Q915219	Briesnitz (Dresden)	Briesnitz	10	None	8017	Briesnitz	1
532:	349099	Q46804	Theuma	Theuma	0	20130			1
533:	349115	Q46760	Heinsdorfergrund	Heinsdorfergrund	0	20153			1
534:	349126	Q649405	Leubnitz (Rosenbach)	Leubnitz_(2)	10	None	20056	Leubnitz (2)	1
535:	349133	Q46764	Limbach (Vogtland)	Limbach_(4)	9	None	13058	Limbach (4)	1
536:	349149	Q46823	Werda	Werda (1)	4	None	13113	Werda (1)	1
537:	349158	Q46773	Neuensalz	Neuensalz	0	20111			1
538:	349166	Q46766	Mühlental	Mühlental	0	19149			1
539:	349176	Q57786	Hormersdorf (Zwönitz)	1=Hormersdorf	12	None			1
540:	349180	Q46784	Neustadt/Vogtl.	Neustadt/Vogtl.	0	13070			1
541:	349182	Q57804	Neukirchen/Erzgeb.	Neukirchen/Erzgeb.	0	14067			1
542:	349186	Q57810	Niederdorf (Sachsen)	Niederdorf	10	None	14070	Niederdorf	1
543:	349187	Q57813	Niederwürschnitz	Niederwürschnitz

657:	733124	Q3952	Plauen	Plauen_(2)	4	None	20082	Plauen (2)	6
658:	738335	Q57921	Lößnitz (Erzgebirge)	Lößnitz_(1)	11	None	21047	Lößnitz (1)	4
659:	741930	Q355595	Adelwitz	Adelwitz	0	30001			1
660:	742600	Q1750755	Kössern	Kössern	0	None			1
661:	742799	Q1630590	Hosterwitz	Hosterwitz	0	8091			1
662:	743533	Q1300749	Ehrenzipfel	Ehrenzipfel	0	21116			1
663:	745983	Q46789	Pöhl	Pöhl	0	20083			1
664:	748722	Q8926	Kleinzschachwitz	Kleinzschachwitz	0	8253			1
665:	761725	Q160688	Doberschau	Doberschau	0	23068			1
666:	767489	Q1319247	Oberhermersdorf	Oberhermersdorf	0	14043			1
667:	767771	Q1587151	Harthau (Oberwiera)	Harthau_(3)	10	None	17035	Harthau (3)	1
668:	770538	Q1531909	Großdeuben	Großdeuben	0	4035			1
669:	778866	Q1815646	Leipziger Vorstadt	Leipziger Vorstadt	0	None			4
670:	782288	Q1789940	Schönbach (Sebnitz)	Schönbach_(5)	8	None	11222	Schönbach (5)	4
671:	785118	Q1416584	Leppersdorf (Wachau)	Leppersdorf	9	None	8109	Leppersdorf	1
672:	791161	Q1826037	Lindenau (Leipzig)	Lindenau_(3)	8	No